### Set up

#### 1. Set  up  accounts and role

In [1]:
#!pip install sagemaker==1.39.0

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201910130520"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
bucket = "aegovan-data"

In [6]:
#trainfile = "s3://{}/aimed/AIMedtrain.json".format(bucket)
#trainfile = "s3://{}/aimed/AIMedFull.json".format(bucket)
trainfile="s3://{}/aimed/AIMedFull_preprocessed.json".format(bucket)

valfile="s3://{}/aimed/AIMedval_preprocessed.json".format(bucket)
# trainfile = "s3://{}/aimed/AIMedtrain_pubmedoverlap.json".format(bucket)
# valfile="s3://{}/aimed/AIMedval_pubmedoverlap.json".format(bucket)
#embeddingfile="s3://{}/embeddings/PubMed-and-PMC-w2v.bin.txt".format(bucket)
#embeddingfile="s3://{}/embeddings/bio_nlp_vec/PubMed-shuffle-win-30.bin.txt".format(bucket)
pretrained_bert="s3://{}/embeddings/bert/".format(bucket)


s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_bert_code".format(bucket)

### Start training

In [7]:
pub_inputs = {
    "train" : trainfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}

In [8]:
BertNetworkFactoryhyperparameters = {
  #  "dataset":"PpiAimedDatasetFactory",
    "dataset":"PpiAimedDatasetPreprocessedFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps" : "4",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20
}

In [9]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [10]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit 0c8536f6df15837d73cd47533e36ed3a1aa5a286
    fix typo


In [11]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
            #  'commit': '58a09e154935248667062a36fdae7d86b86b477c'
             }

In [12]:
hyperparameters = BertNetworkFactoryhyperparameters
inputs = pub_inputs 


In [13]:
hyperparameters

{'accumulation_steps': '4',
 'batchsize': '8',
 'dataset': 'PpiAimedDatasetPreprocessedFactory',
 'earlystoppingpatience': 20,
 'epochs': '1000',
 'learningrate': 1e-05,
 'log-level': 'INFO',
 'network': 'RelationExtractorBioBertFactory',
 'trainfile': 'AIMedFull_preprocessed.json'}

In [14]:
inputs

{'PRETRAINED_BIOBERT': 's3://aegovan-data/embeddings/bert/',
 'train': 's3://aegovan-data/aimed/AIMedFull_preprocessed.json'}

In [15]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
    entry_point='main_train_bert_k_fold.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks','source/trainpipelinesbuilders'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    train_max_run = 60 * 60 * 24 * 4,
                    base_job_name ="aimed-ppi-bert-acc")

In [ ]:
estimator.fit(inputs)

2019-10-27 06:26:48 Starting - Starting the training job...
2019-10-27 06:26:50 Starting - Launching requested ML instances...
2019-10-27 06:27:45 Starting - Preparing the instances for training...
2019-10-27 06:28:42 Downloading - Downloading input data...
2019-10-27 06:28:59 Training - Downloading the training image.........
2019-10-27 06:31:01 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-10-27 06:31:02,767 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-10-27 06:31:02,811 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-10-27 06:31:03,432 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-10-27 06:31:03,793 sagemaker-containers INFO     Module main_train_bert_k_fold does not provide a setup.py. 
Generating setup.py
2019-10-27 06:3

2019-10-27 06:31:17,967 - algorithms.transform_berttext_token_to_index - INFO - Completed TransformBertTextTokenToIndex
2019-10-27 06:31:17,978 - algorithms.transform_berttext_tokenise - INFO - Transforming TransformBertTextTokenise
2019-10-27 06:31:18,511 - algorithms.transform_berttext_tokenise - INFO - Completed TransformBertTextTokenise
2019-10-27 06:31:18,511 - algorithms.transform_berttext_token_to_index - INFO - Transforming TransformBertTextTokenToIndex
2019-10-27 06:31:18,565 - algorithms.transform_berttext_token_to_index - INFO - Completed TransformBertTextTokenToIndex
2019-10-27 06:31:18,566 - algorithms.transform_label_encoder - INFO - Running TransformLabelEncoder
2019-10-27 06:31:18,567 - algorithms.transform_label_encoder - INFO - Complete TransformLabelEncoder
2019-10-27 06:31:18,610 - algorithms.BertTrainInferencePipeline - INFO - Retrieving key learningrate with default .01, found 1e-05
2019-10-27 06:31:18,610 - algorithms.BertTrainInferencePipeline - INFO - Retrievin